In this IPython notebook, we will use tflearn,  a deep learning library build on top of tensorflow to build a deep neural network to classify hand written digits in MNIST dataset.

We start by importing all required modules namely:
- Pandas 
- tflearn
- MNIST dataset from tensorflow.examples

In [4]:
import pandas as pd
import tflearn
from tensorflow.examples.tutorials.mnist import input_data


ModuleNotFoundError: No module named 'tflearn'

In [5]:
!pip install tflearn

     ------------------------------------ 107.3/107.3 kB 693.0 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127290 sha256=1fd0be83722e10598bcdb525d0e0d0e87e646830b73680dcb31f353cafee3830
  Stored in directory: c:\users\computer-_-\appdata\local\pip\cache\wheels\65\9b\15\cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [ ]:
import tensorflow as tf

In [ ]:
tf.reset_default_graph()

Now lets load the dataset from the input folder. We set the one_hot argument as true so that our labels, 0-9, are one hot encoded. 

We will use the input_data.read_data_sets command from tensorflow

In [ ]:
mnist = input_data.read_data_sets('../input/', one_hot=True)

In [ ]:
type(mnist)

Now that we have all our data, we will split it into train, test and validation set. 

Since **mnist** is a tensorflow dataset object, it has inbuilt splits to help us with this process.

In [ ]:

X = mnist.train.images
Y =  mnist.train.labels
testX =  mnist.test.images
testY = mnist.test.labels
valX = mnist.validation.images
valY = mnist.validation.labels
del mnist

Now we shall build our deep MLP to classify the images. The key features of this network are as follows:
- The input layer accepts multiple arrays of length 784. This is because we have flattened our 28x28 sized images into a single array of length 784(=28*28).
- The hidden layers use **relu** activations and regularize all input from last node using **L2** regularisation.
- 30 percent **dropout** is also used to prevent overfilling.
- The output layer uses **softmax**  function to output prediction classes as a one hot encoded vector.
- **Adam** or adaptive momentum optimiser is used.
- Top 5 is used as a evaluation metric.

In [ ]:
input_layer = tflearn.input_data(shape=[None, 784])
dense1 = tflearn.fully_connected(input_layer, 64, activation='relu',
                                 regularizer='L2', weight_decay=0.001)
dropout1 = tflearn.dropout(dense1, 0.7)
dense2 = tflearn.fully_connected(dropout1, 128, activation='relu',
                                 regularizer='L2', weight_decay=0.001)
dropout2 = tflearn.dropout(dense2, 0.7)
dense3 = tflearn.fully_connected(dropout2, 128, activation='relu',
                                 regularizer='L2', weight_decay=0.001)
dropout3 = tflearn.dropout(dense2, 0.7)
softmax = tflearn.fully_connected(dropout3, 10, activation='softmax')

sgd = tflearn.Adam(learning_rate=0.01)
top_k = tflearn.metrics.Top_k(5)
net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,
loss='categorical_crossentropy')

In [ ]:
model = tflearn.DNN(net, tensorboard_verbose=3)


Now that out graph is ready, we shall use the fit methord to feed our training and validation sets. The entire training is run for 2 epochs.

In [ ]:
model.fit(X, Y, n_epoch=2, validation_set=(testX, testY),show_metric=True, run_id="dense_model" )

In [ ]:
del X
del Y
del testX
del testY

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(valX[20].reshape(28,28))

In [ ]:
pd.DataFrame(model.predict(valX[20].reshape(1,784)))

In [ ]:
model.predict_label(valX[20].reshape(1,784))

* As we can see the heighest probability is for 4 which is a correct prediction.

* The **predict_label** methord also returns 4 as most probable followed by a possibility of 9 and then 7.

* Lets evaluate the entire model using the **evaluate** methord of **model**

In [ ]:
y = model.predict(valX)

In [ ]:
# Sanity check
y.shape == valY.shape

In [ ]:
model.evaluate(valX,valY)

As we can see the model performs very well and has an accuracy of 99% on unseen validation set.